In [ ]:
from datasets import Dataset, Image
import numpy as np
import pandas as pd

In [ ]:
ds_csv = pd.read_csv('NLICorpusData_1400.csv')

ds_csv.head()

In [ ]:
# only keep the columns we need, description, index, Scenario
ds_csv = ds_csv[['Description', 'Index', 'Scenario']]

# print the first 5 rows
ds_csv.head()


In [ ]:
block_metadata = pd.read_csv('block_metadata.csv')
# iterate over the rows and create a list of dictionaries

data = []
for index, row in ds_csv.iterrows():
    original_scenario = row['Scenario'][:-4]
    original_scenario = 'v' + original_scenario[-1]

    # remove last 6 characters from scenario
    row['Scenario'] = row['Scenario'][:-7]
    img_config = row['Scenario'][14:]
    row['Scenario'] = row['Scenario'] + '.png'
    row['Scenario'] = 'images/' + row['Scenario']

    # find the block metadata for the current image by comparing the img_config
    block = block_metadata['img_config'] == int(img_config)

    if original_scenario == 'v1':
        block = block & block_metadata['v1'] == 1
        orig_marked_img = 'orig_with_arrows/' + 'Configuration_' + str(img_config).zfill(2) + '_v1.png'

    elif original_scenario == 'v2':
        block = block & block_metadata['v2'] == 1
        orig_marked_img = 'orig_with_arrows/' + 'Configuration_' + str(img_config).zfill(2) + '_v2.png'


    
    # get index of true value
    block_index = block[block == True].index[0]
    answer = block_metadata.iloc[block_index]['block_id']
    synthetic_img = 'synthetic_images/' + 'synthetic_image_' + str(img_config) + '.png'
    
    # 2 digit img_config
    orig_img = 'orig_without_arrows/' + 'Configuration_' + str(img_config).zfill(2) + '.png'


    data.append({'index': row['Index'], 'description': row['Description'], 'scenario': row['Scenario'], 'synthetic_scenario': synthetic_img, 'original_scenario': orig_img, 'original_scenario_marked': orig_marked_img,'answer': answer})

In [ ]:
# print example
data[0]

In [ ]:
def gen():
    for d in data:
        yield d

ds = Dataset.from_generator(gen)
ds = ds.cast_column("scenario", Image())
ds = ds.cast_column("synthetic_scenario", Image())
ds = ds.cast_column("original_scenario", Image())
ds = ds.cast_column("original_scenario_marked", Image())

In [ ]:
ds.push_to_hub('nli_multimodal_ds')